<a href="https://colab.research.google.com/github/EdnTieng/qcomp_mp1/blob/main/problem2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit qiskit-aer qiskit[visualization]

In [ ]:
# YOUNG SHELDONS
# MP Problem 2 — 3-qubit gates


# Imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt
from itertools import product

# Mathematical definitions (for three boolean inputs a,b,c):
# - XOR: a xor b xor c = a ⊕ b ⊕ c (parity = 1 if an odd number of inputs are 1)
# - XNOR: NOT(a ⊕ b ⊕ c) (parity = 1 if an even number of inputs are 1)
# - AND: a ∧ b ∧ c (1 only if all three are 1)
# - NAND: NOT(a ∧ b ∧ c)
# - OR: a ∨ b ∨ c (1 if at least one input is 1)
# - NOR: NOT(a ∨ b ∨ c)
#
# Reversible quantum-circuit strategies used here (3 inputs -> 1 output):
# - For parity (XOR/XNOR) we can compute XOR into the output qubit by sequential CNOTs
# OUT <- OUT ⊕ a ⊕ b ⊕ c; if OUT starts at |0>, after cx from a,b,c it becomes the parity bit.
# For XNOR we invert the parity (X on OUT) to get even-parity detection.
#
# - For multi-input AND: using multi-controlled Toffoli gates (CCX for 2 controls) is the standard approach.
# For three-input AND we can implement a series of CCX gates with an ancilla (or reuse the output as temporary)
# but here we use the OUTPUT qubit as target in a cascade that places (a & b & c) onto OUT.
#
# - For OR and NOR: use De Morgan's law:
# a ∨ b ∨ c = NOT( (NOT a) ∧ (NOT b) ∧ (NOT c) ).
# Thus invert inputs, compute AND, invert result as needed.
#
# Notes about reversibility and ancillas:
# - All quantum operations must be reversible. The simple patterns used here assume we don't need to
# restore intermediate ancilla states for final verification (we restore inverted inputs when needed).
# - The circuits use 4 qubits total: q0 (A), q1 (B), q2 (C), q3 (OUT). OUT is initialised to |0>.

def validate_truth_table(tt):
    """
    Validate a 3-input truth table.
    tt: list of tuples/lists (a,b,c,out)
    """
    if not isinstance(tt, (list, tuple)):
        return False
    if len(tt) != 8:
        return False
    seen_inputs = set()
    for row in tt:
        if len(row) != 4:
            return False
        if any(bit not in (0,1) for bit in row):
            return False
        seen_inputs.add((row[0], row[1], row[2]))

    all_inputs = set(product([0,1], repeat=3))
    return seen_inputs == all_inputs

_example_tt = [(a,b,c, 0) for (a,b,c) in product([0,1], repeat=3)]
assert validate_truth_table(_example_tt)


# Simulate helper functions
sim = AerSimulator()

def run_circuit_and_readout(qc, shots=1024):
"""
Execute qc on the aer_simulator and return histogram of measurement results (classical bits as string)
Expect the measurement to have 4 classical bits in order [q3 q2 q1 q0] when measured that way.
"""
    qc_t = transpile(qc, sim)
    result = sim.run(qc_t, shots=shots).result()
    return result.get_counts()


def measure_output_bit(qc, shots=1024):
"""
Utility that appends measurements to the circuit for all qubits, runs it, and extracts the OUT qubit (q3) result.
Returns probability of OUT=1 and OUT=0 (as dictionary)
"""
    qc_meas = qc.copy()
    creg = ClassicalRegister(4, 'c')
    qc_meas.add_register(creg)
    qc_meas.measure([3,2,1,0], [3,2,1,0])

    counts = run_circuit_and_readout(qc_meas, shots)
    out1 = sum(cnt for bitstr, cnt in counts.items() if bitstr[0] == '1')
    total = sum(counts.values())
    return {"0": (total - out1) / total, "1": out1 / total}


# Build XNOR circuit gate
def build_xnor_circuit():
"""
XNOR output = NOT(a xor b xor c) — returns 1 if even number of ones among inputs.
Implementation: OUT <- a ⊕ b ⊕ c, then X on OUT to invert parity.
"""
    qc = QuantumCircuit(4)
    qc.cx(0, 3)
    qc.cx(1, 3)
    qc.cx(2, 3)
    qc.x(3)
    return qc

# Build NAND circuit gate
def build_nand_circuit():
"""
NAND = NOT(a AND b AND c)
Implementation: use cascade of CCX gates to compute AND into OUT, then invert OUT.
Note: this approach uses OUT as a temporary for the intermediate AND calculation.
"""
    qc = QuantumCircuit(5)
    qc.ccx(0, 1, 4)
    qc.ccx(4, 2, 3)
    qc.x(3)
    qc.ccx(0, 1, 4)

    return qc

# Build OR circuit gate
def build_or_circuit():
"""
OR = a ∨ b ∨ c = NOT( (NOT a) ∧ (NOT b) ∧ (NOT c) )
We will implement by inverting inputs, computing AND into OUT (with ancilla), then inverting OUT.
"""
    qc = QuantumCircuit(5)
    qc.x(0); qc.x(1); qc.x(2)
    qc.ccx(0, 1, 4)
    qc.ccx(4, 2, 3)
    qc.x(3)
    qc.ccx(0, 1, 4)
    qc.x(0); qc.x(1); qc.x(2)

    return qc

# Build NOR circuit gate
def build_nor_circuit():
"""
NOR = NOT(OR) = NOT(a ∨ b ∨ c)
Using De Morgan: compute AND of inverted inputs -> that's OR, NOR is complement of OR.
One can implement NOR by inverting inputs, computing AND into OUT, then NOT the OUT? Careful with double negation.
We'll implement NOR directly by computing OR (as above) and then inverting OUT to get NOR.
"""
    qc = build_or_circuit()
    qc.x(3)
    return qc


# Classical reference functions
def classical_xnor(a,b,c): return int(not (a ^ b ^ c))
def classical_nand(a,b,c): return int(not (a & b & c))
def classical_or(a,b,c):   return int(a | b | c)
def classical_nor(a,b,c):  return int(not (a | b | c))

# Input preparation
def prepare_inputs_on_circuit(base_qc, a,b,c, nqubits=None):
"""
Returns a new circuit where q0..q2 are set to values a,b,c (via X gates if bit=1), and other qubits remain.
base_qc: expects the gate circuit that assumes inputs already prepared. We'll create a new circuit with the same
number of qubits and copy the base operations after preparing inputs.
"""
    if nqubits is None:
        nqubits = base_qc.num_qubits

    qc = QuantumCircuit(nqubits)

    if a: qc.x(0)
    if b: qc.x(1)
    if c: qc.x(2)

    qc.compose(base_qc, inplace=True)
    return qc


# Verification of circuits
def verify_circuit(builder, classical_func):
"""
builder: function returning a QuantumCircuit
classical_func: function(a,b,c) -> expected output bit
use_ancilla: if True, the builder returns a 5-qubit circuit (with ancilla at index 4)
Returns a list of (a,b,c, out_qiskit, out_expected) and a Qiskit circuit for one example.
"""
    results = []
    base_qc = builder()
    nqubits = base_qc.num_qubits

    for a,b,c in product([0,1], repeat=3):
        qc_prep = prepare_inputs_on_circuit(base_qc, a,b,c, nqubits)
        probs = measure_output_bit(qc_prep, shots=2048)
        out_q = 1 if probs["1"] > probs["0"] else 0
        out_expected = classical_func(a,b,c)

        results.append((a,b,c,out_q,out_expected,probs))
    return results


# Run verification
print("Verifying XNOR...")
xnor_results = verify_circuit(build_xnor_circuit, classical_xnor)

print("Verifying NAND...")
nand_results = verify_circuit(build_nand_circuit, classical_nand)

print("Verifying OR...")
or_results = verify_circuit(build_or_circuit, classical_or)

print("Verifying NOR...")
nor_results = verify_circuit(build_nor_circuit, classical_nor)


# Printing of results
def print_results_table(results, title):
    print("\n" + title)
    print("a b c | qiskit | expected | P(out=1)")
    print("-" * 45)
    for a,b,c,out_q,out_expected,probs in results:
        print(f"{a} {b} {c} |   {out_q}    |     {out_expected}    | {probs['1']:.3f}")

print_results_table(xnor_results, "3-QUBIT XNOR")
print_results_table(nand_results, "3-QUBIT NAND")
print_results_table(or_results,   "3-QUBIT OR")
print_results_table(nor_results,  "3-QUBIT NOR")


Verifying XNOR...
Verifying NAND...
Verifying OR...
Verifying NOR...

3-QUBIT XNOR
a b c | qiskit | expected | P(out=1)
---------------------------------------------
0 0 0 |   1    |     1    | 1.000
0 0 1 |   0    |     0    | 0.000
0 1 0 |   0    |     0    | 0.000
0 1 1 |   1    |     1    | 1.000
1 0 0 |   0    |     0    | 0.000
1 0 1 |   1    |     1    | 1.000
1 1 0 |   1    |     1    | 1.000
1 1 1 |   0    |     0    | 0.000

3-QUBIT NAND
a b c | qiskit | expected | P(out=1)
---------------------------------------------
0 0 0 |   1    |     1    | 1.000
0 0 1 |   1    |     1    | 1.000
0 1 0 |   1    |     1    | 1.000
0 1 1 |   1    |     1    | 1.000
1 0 0 |   1    |     1    | 1.000
1 0 1 |   1    |     1    | 1.000
1 1 0 |   1    |     1    | 1.000
1 1 1 |   0    |     0    | 0.000

3-QUBIT OR
a b c | qiskit | expected | P(out=1)
---------------------------------------------
0 0 0 |   0    |     0    | 0.000
0 0 1 |   1    |     1    | 1.000
0 1 0 |   1    |     1    | 1.